### Создание DataSet / Парсинг по разделам journal.ksk.expert

In [13]:
import requests
from bs4 import BeautifulSoup
import json
from pprint import pprint
import re

session = requests.Session()
#Ссылка на авторизацию POST запрос
link_auth = 'https://journal.ksk.expert/login/?login=yes'
#Ссылка на раздел который парсим
URL = 'https://journal.ksk.expert/articles/hr-keysy/'

HEADERS = {
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Mobile Safari/537.36',
    'accept': '*/*'
}

data_auth = {
    'backurl': '/login/',
    'AUTH_FORM': 'Y',
    'TYPE': 'AUTH',
    'USER_LOGIN': 'maltseivan',
    'USER_PASSWORD': '*********',
    'Login': 'Войти'
}

responce = session.post(link_auth, data=data_auth, headers=HEADERS)
# Ответ сервера на POST запрос с авторизацией: print(responce)

#data = Хранятся полученные статьи
data_list = {}

def get_html(url, params=None):
    r = session.get(url, headers=HEADERS, params=params)
    return r
def get_content(html):
    #Парсер ссылок раздела
    soup = BeautifulSoup(html, 'html.parser')
    link = soup.find_all('a', class_='link-article')
    cat = soup.find('h1', class_='ch-title').text
    #Раздел в котором парсим pprint(cat)
    
    for elem in link:
        link_href_attr = elem.get("href")
        urlink = 'https://journal.ksk.expert' + link_href_attr
        
        #Парсер статьи по ссылкам
        def get_html_article(urlink, params=None):
            r_text = session.get(urlink, headers=HEADERS, params=params)
            return r_text

        def get_content_article(html):
            soup = BeautifulSoup(html, 'html.parser')
            chapter_article = soup.find('h1', class_='ch-title').text
            content_article = soup.find_all('div', class_='contents mb-60')
            #Название статей в разделе pprint(chapter_article)

            for elem in content_article:
                index_text = elem.text
                #Контентая часть статьи pprint(index_text[0])
                data_list[chapter_article] = [index_text]
                
                with open(cat + '.json', 'w', encoding='utf-8') as file:
                    json.dump(data_list, file, indent=3, ensure_ascii=False)

        def parse_article():
            result = get_html_article(urlink)
            if result.status_code == 200:
                get_content_article(result.text)
            else:
                print('Не могу получить страницу!')
         
        parse_article();
        

def parse():
    result = get_html(URL)
    if result.status_code == 200:
        get_content(result.text)
    else:
        print('Не могу получить страницу!')


parse();


In [1]:
import json
from pprint import pprint

data = {}

with open("HR-кейсы.json", "r", encoding='utf-8') as file:
    data['HR-кейсы'] = json.load(file)
    
with open("HR-тесты.json", "r", encoding='utf-8') as file:
    data['HR-тесты'] = json.load(file) 
    
with open("HR-юмор.json", "r", encoding='utf-8') as file:
    data['HR-юмор'] = json.load(file) 
    
with open("Автоматизация HR.json", "r", encoding='utf-8') as file:
    data['Автоматизация HR'] = json.load(file) 
    
with open("Адаптация.json", "r", encoding='utf-8') as file:
    data['Адаптация'] = json.load(file)
    
with open("Бизнес.json", "r", encoding='utf-8') as file:
    data['Бизнес'] = json.load(file)

with open("Корпоративная культура.json", "r", encoding='utf-8') as file:
    data['Корпоративная культура'] = json.load(file)
    
with open("Мотивация.json", "r", encoding='utf-8') as file:
    data['Мотивация'] = json.load(file)  
    
with open("Обучение персонала.json", "r", encoding='utf-8') as file:
    data['Обучение персонала'] = json.load(file) 
    
with open("Рекрутмент.json", "r", encoding='utf-8') as file:
    data['Рекрутмент'] = json.load(file) 

with open("Трудовое законодательство.json", "r", encoding='utf-8') as file:
    data['Трудовое законодательство'] = json.load(file) 

with open("Управление персоналом.json", "r", encoding='utf-8') as file:
    data['Управление персоналом'] = json.load(file) 
    
with open('data.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=3, ensure_ascii=False)
    

### Создание DataFrame

In [2]:
from tqdm import tqdm
import pandas as pd

category = []
name_article = []
article = []

for elem in data:
    for value in data[elem]:
        for text in data[elem][value]:
            category.append(elem)
            name_article.append(value)
            article.append(text)
df_data = {
    'Категория': category,
    'Название статьи': name_article,
    'Статья': article
}

df = pd.DataFrame(df_data)
df

,Категория,Название статьи,Статья
0,HR-кейсы,Как повысить лояльность сотрудников,"\n\nЗадумываетесь, как увеличить благонадежнос..."
1,HR-кейсы,Excel для эйчаров: три кейса,"\n\n\t\t\t\t об авторе:\n\nНиколай Павлов, ..."
2,HR-кейсы,Может ли психолог быть эйчаром?,"\n\nОб авторе:\n\n\n\n\n\nАндрей Коновалов, ос..."
3,HR-кейсы,HR‑фишка #2. Как собирать рекомендации надёжнее,\n\r\n\t Вы когда-нибудь давали рекомендации н...
4,HR-кейсы,Как повысить отклик на вакансию,"\n\r\n\t Кандидат прислал интересное резюме, а..."
...,...,...,...
577,Управление персоналом,В кого и почему превращаются эйчары,"\n\n\n\n\r\n\t Гусеница становится куколкой, к..."
578,Управление персоналом,"Социальная политика компании: введение, термин...",\n\nОб авторе:\n\n\n\n\n\n\nНовикова Марина Ль...
579,Управление персоналом,Чем же управляет HR-менеджер?,\n\r\n\t Пришедшее из английского языка слово ...
580,Управление персоналом,Как оптимизировать работу подразделения (униве...,"\n\r\n\t В компании, численность которой превы..."


### Обработка текста методоми NLP 

In [3]:
stops = []
with open('stop-ru.txt', 'r', encoding='utf-8') as f:
    for word in f:
        stops.append(word.strip())
       #print(word.strip())

In [4]:
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import re

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

data_lem = []

for lem_text in article:
    data_lem.append(lemmatize(lem_text))

print(data_lem[0])
    

['задумываться', 'увеличить', 'благонадёжность', 'преданность', 'верность', 'сотрудник', 'ваш', 'организация', 'это', 'несложно', 'достаточно', 'разработать', 'некоторый', 'организационный', 'мероприятие', 'для', 'бизнес', 'лояльность', 'сотрудник', 'являться', 'полезный', 'явление', 'важно', 'сотрудник', 'готовый', 'работать', 'результат', 'понадобиться', 'идти', 'уступка', 'в', 'копенгагенский', 'школа', 'маркетинг', 'вычислить', 'лояльность', 'работник', 'увеличить', 'лояльность', 'клиент', 'подняться', 'при', 'лояльность', 'клиент', 'расти', 'прибыль', 'организация', 'следующий', 'квартал', 'увеличиться', 'согласно', 'исследование', 'половина', 'сотрудник', 'средний', 'компания', 'искать', 'другой', 'возможность', 'трудоустройство', 'и', 'неудивительный', 'многие', 'компания', 'ежегодно', 'терять', 'около', 'треть', 'свой', 'сотрудник', 'с', 'такой', 'цифра', 'лояльность', 'сотрудник', 'казаться', 'дело', 'прошлое', 'согласно', 'исследование', 'провести', 'центр', 'американский', '

In [6]:
df_data = {
    'Категория': category,
    'Название статьи': name_article,
    'Статья': article,
    'Лематизация текста': data_lem
}

df = pd.DataFrame(df_data)
df

,Категория,Название статьи,Статья,Лематизация текста
0,HR-кейсы,Как повысить лояльность сотрудников,"\n\nЗадумываетесь, как увеличить благонадежнос...","[задумываться, увеличить, благонадёжность, пре..."
1,HR-кейсы,Excel для эйчаров: три кейса,"\n\n\t\t\t\t об авторе:\n\nНиколай Павлов, ...","[автор, николай, павлов, профессиональный, тре..."
2,HR-кейсы,Может ли психолог быть эйчаром?,"\n\nОб авторе:\n\n\n\n\n\nАндрей Коновалов, ос...","[о, автор, андрей, коновалов, основатель, ‑, п..."
3,HR-кейсы,HR‑фишка #2. Как собирать рекомендации надёжнее,\n\r\n\t Вы когда-нибудь давали рекомендации н...,"[вы, давать, рекомендация, сотрудник, коллега,..."
4,HR-кейсы,Как повысить отклик на вакансию,"\n\r\n\t Кандидат прислал интересное резюме, а...","[кандидат, прислать, интересный, резюме, ваш, ..."
...,...,...,...,...
577,Управление персоналом,В кого и почему превращаются эйчары,"\n\n\n\n\r\n\t Гусеница становится куколкой, к...","[гусеница, становиться, куколка, куколка, бабо..."
578,Управление персоналом,"Социальная политика компании: введение, термин...",\n\nОб авторе:\n\n\n\n\n\n\nНовикова Марина Ль...,"[о, автор, новиков, марина, львович, бизнес, к..."
579,Управление персоналом,Чем же управляет HR-менеджер?,\n\r\n\t Пришедшее из английского языка слово ...,"[прийти, английский, язык, слово, «менеджер», ..."
580,Управление персоналом,Как оптимизировать работу подразделения (униве...,"\n\r\n\t В компании, численность которой превы...","[в, компания, численность, который, превышать,..."


### Пример разбора одной статьи с вхождением слов  + Топ_10 слов статьи

In [7]:
from nltk import word_tokenize
from nltk.probability import FreqDist

text_one = data_lem[0]
fdist = FreqDist(text_one)
#pprint(fdist)

In [8]:
fdist.most_common(10)

[('сотрудник', 36),
 ('лояльность', 13),
 ('который', 11),
 ('свой', 9),
 ('компания', 8),
 ('это', 7),
 ('работа', 7),
 ('организация', 5),
 ('рабочий', 5),
 ('друг', 5)]

In [10]:
#Результат после Токенизации, Лематизации обратно в нормальный текст из списка в качестве разделителя пробел
article_result = []
for alem_article in data_lem:
    article_result.append(" ".join(alem_article))
print(article_result[-15])

частный охранный агентство «омег » существовать год это время успеть завоевать репутация профессиональный надёжный партнёр специфика работа рынок детективный охранный услуга предполагать высокий степень конфиденциальность поэтому главный принцип деятельность компания подбор профессиональный кадр гарантия соблюдение тайна обращение клиент в компания разработать специальный стандарт внутренний политика который подробно регламентировать взаимодействие клиент порядок работа предоставить они информация всё сотрудник приступать работа подписывать соглашение неразглашение коммерческий тайна работник допустить однократный нарушение этот правило подвергаться увольнение жёсткость такой порядок объясняться сотрудник компания долг служба получать доступ огромный массив личный коммерческий информация клиент несанкционированный разглашение этот сведение мочь причинить значительный ущерб оба сторона однажды «омег » прийти запрос орган мвд требование предоставить ряд документ касаться один сотрудник к

In [11]:
df_data = {
    'Категория': category,
    'Название статьи': name_article,
    'Статья': article,
    'Лематизация текста': data_lem,
    'Очищенный текст': article_result
}

df = pd.DataFrame(df_data)
df

,Категория,Название статьи,Статья,Лематизация текста,Очищенный текст
0,HR-кейсы,Как повысить лояльность сотрудников,"\n\nЗадумываетесь, как увеличить благонадежнос...","[задумываться, увеличить, благонадёжность, пре...",задумываться увеличить благонадёжность преданн...
1,HR-кейсы,Excel для эйчаров: три кейса,"\n\n\t\t\t\t об авторе:\n\nНиколай Павлов, ...","[автор, николай, павлов, профессиональный, тре...",автор николай павлов профессиональный тренер п...
2,HR-кейсы,Может ли психолог быть эйчаром?,"\n\nОб авторе:\n\n\n\n\n\nАндрей Коновалов, ос...","[о, автор, андрей, коновалов, основатель, ‑, п...",о автор андрей коновалов основатель ‑ психолог...
3,HR-кейсы,HR‑фишка #2. Как собирать рекомендации надёжнее,\n\r\n\t Вы когда-нибудь давали рекомендации н...,"[вы, давать, рекомендация, сотрудник, коллега,...",вы давать рекомендация сотрудник коллега если ...
4,HR-кейсы,Как повысить отклик на вакансию,"\n\r\n\t Кандидат прислал интересное резюме, а...","[кандидат, прислать, интересный, резюме, ваш, ...",кандидат прислать интересный резюме ваш ответн...
...,...,...,...,...,...
577,Управление персоналом,В кого и почему превращаются эйчары,"\n\n\n\n\r\n\t Гусеница становится куколкой, к...","[гусеница, становиться, куколка, куколка, бабо...",гусеница становиться куколка куколка бабочка г...
578,Управление персоналом,"Социальная политика компании: введение, термин...",\n\nОб авторе:\n\n\n\n\n\n\nНовикова Марина Ль...,"[о, автор, новиков, марина, львович, бизнес, к...",о автор новиков марина львович бизнес консульт...
579,Управление персоналом,Чем же управляет HR-менеджер?,\n\r\n\t Пришедшее из английского языка слово ...,"[прийти, английский, язык, слово, «менеджер», ...",прийти английский язык слово «менеджер» означа...
580,Управление персоналом,Как оптимизировать работу подразделения (униве...,"\n\r\n\t В компании, численность которой превы...","[в, компания, численность, который, превышать,...",в компания численность который превышать один ...


### Векторизация частота вxождения слов в статьи

In [12]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv = CountVectorizer(max_df=0.8, min_df=2, stop_words=stops)
print(article_result[50])
count = cv.fit(article_result)
count = cv.transform(article_result)

df_vec = pd.DataFrame(count.toarray(), columns = cv.get_feature_names())
df_vec

ежегодно американский организация сам понимать тратить миллион доллар мероприятие адаптация мероприятие адаптация ориентация интеграция работник – назвать это наверное единственный форма тренинг который совершенно оцениваться компания предмет рентабельность инвестиция анкета заполнить фотография пропуск сделать страховой пособие согласовать видеопытка история компания продукт услуга возможно достоинство выдержать настать момент новый работник готовый приступить работа и получать стать часть организация работать новичок свой первый рабочий день слышать правило основание увольнение абсурд один трата деньга но способ хороший доказать первый день «информационный мусор» мероприятие адаптация малоэффективный я основываться мой собственный исследование книга маркус бакингэма курт коффмана «сперва забыть правилах» работа компания вместо информационный мусор попробовать нижеследующий мероприятие адаптация – это пособие формальность политика компания мероприятие адаптация должный отражать миссия

,аббревиатура,абзац,абитуриент,абонемент,абонентский,абсолютный,абстрагироваться,абстрактный,абсурд,аванс,...,ясно,ясность,ясный,ячейка,ящик,ёлка,ёлочный,ёмкий,ёмко,ёмкость
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
578,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
580,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Тематическое моделирование

In [13]:
from sklearn.decomposition import LatentDirichletAllocation
import random

LDA = LatentDirichletAllocation(n_components=12, random_state=42)
LDA.fit(count)
first_topic = LDA.components_[0]

print('Топ 10 слов для наиболее подходяшей темы :)')
top_topic_words = first_topic.argsort()[-10:]
for i in top_topic_words:
    print(cv.get_feature_names()[i])

Топ 10 слов для наиболее подходяшей темы :)
руководитель
ситуация
проблема
коллега
работодатель
директор
менеджер
отпуск
рабочий
персонал


In [14]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 15 words for topic #{i}:')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

Top 15 words for topic #0:
['начальник', 'уволить', 'коллектив', 'офис', 'неделя', 'руководитель', 'ситуация', 'проблема', 'коллега', 'работодатель', 'директор', 'менеджер', 'отпуск', 'рабочий', 'персонал']


Top 15 words for topic #1:
['ситуация', 'резюме', 'информация', 'опыт', 'работодатель', 'агентство', 'поиск', 'руководитель', 'собеседование', 'подбор', 'соискатель', 'специалист', 'рекрутер', 'вакансия', 'кандидат']


Top 15 words for topic #2:
['качество', 'здоровый', 'специалист', 'рынок', 'специализация', 'возможность', 'сегмент', 'услуга', 'опцион', 'стандарт', 'акция', 'персонал', 'программа', 'агентство', 'менеджер']


Top 15 words for topic #3:
['увольнение', 'закон', 'рабочий', 'срок', 'условие', 'организация', 'отпуск', 'тк', 'труд', 'ст', 'рф', 'договор', 'работодатель', 'работник', 'трудовой']


Top 15 words for topic #4:
['директор', 'результат', 'поощрение', 'рейтинг', 'бизнес', 'цель', 'бренд', 'уровень', 'высокий', 'профессиональный', 'менеджер', 'фронт', 'клиент',

### Присвоение темы статьи после тематического моделирования

In [15]:
topic_values = LDA.transform(count)
topic_values.shape

df_vec_theme = df_vec

df_vec_theme['Тема статьи'] = topic_values.argmax(axis=1)
df_vec_theme

,аббревиатура,абзац,абитуриент,абонемент,абонентский,абсолютный,абстрагироваться,абстрактный,абсурд,аванс,...,ясность,ясный,ячейка,ящик,ёлка,ёлочный,ёмкий,ёмко,ёмкость,Тема статьи
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
1,0,0,0,0,0,1,0,0,0,0,...,0,0,6,0,0,0,0,0,0,11
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,5
578,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,8
579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
580,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11


### Выбор и обучение модели ML, случайный лес, деление Data на X_train и y_train

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split

df_data_word = df_vec_theme.drop(columns = 'Тема статьи')

word = df_data_word.columns

X = df_vec_theme.loc[:, word]
y = df_vec_theme.loc[:, ['Тема статьи']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=85, train_size = 0.3)

In [17]:
X_test.head(1)

,аббревиатура,абзац,абитуриент,абонемент,абонентский,абсолютный,абстрагироваться,абстрактный,абсурд,аванс,...,ясно,ясность,ясный,ячейка,ящик,ёлка,ёлочный,ёмкий,ёмко,ёмкость
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y_test

,Тема статьи
42,1
34,1
289,1
225,9
52,1
...,...
547,0
487,1
82,7
58,0


### Использование случайного леса для регрессии 

In [18]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Обучение алгоритму

In [19]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

C:\Users\IVANMA~1\AppData\Local\Temp/ipykernel_14180/2946511631.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


In [20]:
y_pred

array([5.75, 4.45, 4.5 , 8.3 , 5.4 , 3.2 , 7.5 , 4.05, 5.35, 7.95, 6.1 ,
       7.9 , 5.95, 4.55, 6.  , 4.75, 7.6 , 8.8 , 5.9 , 3.25, 4.85, 2.9 ,
       7.9 , 2.55, 6.2 , 6.2 , 6.25, 4.3 , 4.5 , 6.35, 6.1 , 4.6 , 2.85,
       2.95, 4.4 , 6.6 , 2.35, 4.75, 3.5 , 6.35, 6.25, 4.75, 2.8 , 1.2 ,
       6.5 , 7.3 , 7.75, 5.  , 7.7 , 5.5 , 6.15, 5.75, 4.  , 2.65, 4.15,
       4.25, 6.85, 7.3 , 4.1 , 6.1 , 2.8 , 7.05, 7.1 , 7.3 , 9.35, 7.5 ,
       9.7 , 4.75, 3.7 , 5.65, 4.15, 6.1 , 6.65, 6.15, 2.25, 7.7 , 3.35,
       7.1 , 6.55, 7.35, 3.4 , 8.1 , 3.95, 6.35, 7.75, 4.2 , 4.  , 6.3 ,
       2.4 , 6.35, 7.75, 4.05, 9.8 , 6.35, 2.95, 3.1 , 7.05, 4.5 , 6.6 ,
       6.85, 5.25, 5.85, 6.  , 5.8 , 3.8 , 4.05, 5.4 , 2.15, 3.4 , 5.  ,
       6.05, 3.05, 7.5 , 5.6 , 4.15, 6.75, 6.95, 6.8 , 3.05, 5.95, 7.4 ,
       7.95, 6.8 , 3.75, 3.3 , 6.8 , 3.25, 5.55, 5.15, 5.7 , 2.9 , 4.1 ,
       8.5 , 4.15, 2.8 , 3.8 , 4.25, 5.05, 5.5 , 8.9 , 4.75, 4.3 , 3.15,
       2.75, 2.6 , 7.6 , 4.2 , 6.05, 3.05, 5.95, 6.

### Оценка алгоритма

In [21]:
from sklearn import metrics

print('Средняя абсолютная ошибка:', metrics.mean_absolute_error(y_test, y_pred))
print('Средняя квадратическая ошибка:', metrics.mean_squared_error(y_test, y_pred))
print('Среднеквадратичная ошибка:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Средняя абсолютная ошибка: 2.9025735294117645
Средняя квадратическая ошибка: 11.5931556372549
Среднеквадратичная ошибка: 3.40487233787919


### Класификация Random Forest Classifier

In [22]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)
y_predict

C:\Users\IVANMA~1\AppData\Local\Temp/ipykernel_14180/3011062360.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


array([ 1,  3,  1,  5, 11,  3, 11,  3,  3, 11,  5, 11,  3,  1,  5,  1,  3,
        9,  9,  3,  9,  1, 11,  1,  9,  1,  9,  3,  1,  3, 11,  1,  1,  3,
        3,  3,  3,  5,  1,  3,  9,  1,  3,  1,  9, 11, 11,  3, 11,  1, 11,
        3,  1,  3,  3,  5,  9, 11,  1,  9,  3,  9,  3, 11,  5,  9, 11,  3,
        9,  3,  1,  5,  3,  3,  1,  3,  1,  9,  9,  5,  3,  9,  1,  5, 11,
        3,  9,  5,  1,  5,  5,  9, 11,  9,  3,  3,  9,  9,  0, 11,  5, 11,
       11,  9,  1,  1,  1,  3,  1,  5,  9,  3,  5,  1,  1, 11, 11, 11,  1,
        3,  9, 11, 11,  3,  1, 11,  1,  9,  9,  3,  3,  9, 11,  9,  9,  3,
        1,  5,  9,  1,  1,  1,  5,  3,  1,  9,  1,  9,  1,  1,  5,  9,  1,
       11,  1,  9, 11,  3,  3,  5, 11,  1,  1,  9,  3,  9,  9,  3,  9, 11,
       11, 11, 11,  3,  1, 11,  1,  1,  3,  9,  1,  9,  1, 11,  9,  1,  5,
        3,  9,  9,  5,  1,  1, 11,  1,  9,  3, 11,  9,  3,  3,  9,  1,  1,
        9,  3,  1,  9,  9,  9,  9,  1,  9,  3,  5,  1,  5,  9,  9, 11,  3,
        9,  1, 11,  9,  1

In [23]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_test, y_test)
y_predct = clf.predict(X_test)
y_predct

C:\Users\IVANMA~1\AppData\Local\Temp/ipykernel_14180/2030219447.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_test, y_test)


array([ 1,  1,  1,  9,  1,  7, 11,  3,  3, 10,  5,  6,  3,  9,  1,  1, 11,
        9, 11,  3,  0,  1, 11,  1, 10, 11, 10,  5,  1,  8, 11,  1,  1,  3,
        3, 11,  7, 11,  4,  0,  6,  1,  3,  1,  9, 11,  5,  7,  5,  9, 10,
        4,  7,  1,  3,  5,  5, 11,  1,  9,  3,  5,  5, 11, 11,  9, 11,  3,
        0,  9,  9,  1,  3,  9,  1,  7, 10,  5,  9,  9,  0,  9,  1,  5,  9,
        8,  9,  5,  1,  9, 10,  7, 11,  0,  3,  6,  9,  0,  9,  9,  5,  1,
       11,  6,  1,  5,  2,  7,  1,  1,  9,  3,  9,  5,  0, 11,  5, 11,  7,
        3,  5, 11,  1,  0,  1,  1,  1,  0,  5,  9,  8,  0, 11,  9,  9,  0,
        2,  5,  9, 11,  1,  7,  1,  7, 11,  9,  1,  9,  8,  4,  5, 10,  2,
        9,  1,  5,  4,  5,  3,  5,  5,  5,  1,  1,  5,  8,  9,  6,  8,  2,
        5, 11, 11,  3,  5,  6,  1, 10,  3,  7,  1,  0,  1,  5,  9,  4, 11,
        0,  9, 11,  5,  5,  1, 11,  0, 11,  0, 11,  9,  3,  6,  2,  0,  1,
        1,  3, 11,  9,  8,  9,  9,  1,  4,  3,  5, 11,  8,  9,  9, 10,  9,
        7,  1, 10,  5,  1

### Проверка на сырых данных

In [26]:
post = """ежегодно американский организация сам понимать тратить миллион доллар мероприятие адаптация мероприятие адаптация ориентация интеграция работник – назвать это наверное единственный форма тренинг который совершенно оцениваться компания предмет рентабельность инвестиция анкета заполнить фотография пропуск сделать страховой пособие согласовать видеопытка история компания продукт услуга возможно достоинство выдержать настать момент новый работник готовый приступить работа и получать стать часть организация работать новичок свой первый рабочий день слышать правило основание увольнение абсурд один трата деньга но способ хороший доказать первый день «информационный мусор» мероприятие адаптация малоэффективный я основываться мой собственный исследование книга маркус бакингэма курт коффмана «сперва забыть правилах» работа компания вместо информационный мусор попробовать нижеследующий мероприятие адаптация – это пособие формальность политика компания мероприятие адаптация должный отражать миссия ценность культура организация не важно семейный бизнес многонациональный корпорация человек хотеть часть тот организация чей футболка гордость одеть пляж компания пойти далёкий в первый рабочий день весь семья новый сотрудник получать цветок футболка который компания присылать он домой фирма город маклина штат виржиния свой новобранец отсылать специальный учебный лагерь давать возможность влиться новый коллектив мероприятие адаптация длиться несколько день включать технический подготовка тим билдинга пусть новобранец быстрый начать работать дать задание использовать свежий энергия ресурс воодушевление переутомиться команда ваш человек ждать новый человек новый сила приступить работа применять мероприятие адаптация пошаговый подход в первый день организовать небольшой встреча затем неделя встретиться продолжать проводить небольшой встреча течение первый три четыре неделя составить график организовать встреча различный отдел часть организация использовать мероприятие адаптация один возможность новый работник узнать компания хороший в дальнейший пусть сотрудник возможность узнавать различный мероприятие фирма человек приходить компания уходить менеджер как скорее привлекать адаптация большинство приписывать отдел мероприятие эффективный реализоваться один отдел согласиться новый работник скорее быть уделять внимание тот говорить менеджер слушать кто отдел когда менеджер ответственный успех свой новый работник это сильно влиять удачный начинание последний дать новичок такой задание который смочь выполнить пара неделя день многий нравиться получение должность позволять выполнять настоящий работа кстати согласно исследование процент добровольно сменить работа сделать это хотеть чувствовать ответственность свой труд мочь влиять человек уходить небольшой организация хотеть чувствовать просто маленький винтик гигантский машина я постоянно слышать интервью увольнение такой фраза «я уходить менее крупный организация действительно смочь изменить» отслеживать влиять улучшение мероприятие адаптация работник текучесть рабочий сила если вышеперечисленный шаг правильно выполняться прийтись часто повторять"""

### Функция преобразования слов в числа и предсказывание темы

In [27]:
def theme_article(post):
    
    global post_NLP
    post_NLP = lemmatize(post)
    
    article_result_new = []
    article_result_new = article_result
    article_result_new.append(" ".join(post_NLP))
    cv_word = CountVectorizer(max_df=0.8, min_df=2, stop_words=stops)
    count_word = cv_word.fit(article_result_new)
    count_word = cv_word.transform(article_result_new)
    df_vec_word = pd.DataFrame(count_word.toarray(), columns = cv_word.get_feature_names())

    count_new_word_article = df_vec_word.iloc[-1]
    data_word_count = []
    
    for elem in count_new_word_article:
        data_word_count.append(elem)

    len(data_word_count)
    delete = len(data_word_count) - 11816
    del data_word_count[1:delete]
    clf.predict([data_word_count])
    theme_article = clf.predict([data_word_count])
    print(theme_article[0])

theme_article(post);

9


### API 

In [ ]:
class ThemeArticle:
    def theme_article(post):
    
        global post_NLP
        post_NLP = lemmatize(post)

        article_result_new = []
        article_result_new = article_result
        article_result_new.append(" ".join(post_NLP))
        cv_word = CountVectorizer(max_df=0.8, min_df=2, stop_words=stops)
        count_word = cv_word.fit(article_result_new)
        count_word = cv_word.transform(article_result_new)
        df_vec_word = pd.DataFrame(count_word.toarray(), columns = cv_word.get_feature_names())

        count_new_word_article = df_vec_word.iloc[-1]
        data_word_count = []

        for elem in count_new_word_article:
            data_word_count.append(elem)

        len(data_word_count)
        delete = len(data_word_count) - 11816
        del data_word_count[1:delete]
        clf.predict([data_word_count])
        theme_article = clf.predict([data_word_count])
        print(theme_article[0])

    theme_article(post);